In [57]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt

In [62]:
# Load first 500 rows from csv
df1 = pd.read_csv('../TH_Data/500_rows.csv', index_col=0)

# Load second 500 rows from csv
df2 = pd.read_csv('../TH_Data/500_2_rows.csv', index_col=0)

# Concatenate dfs
frames = [df1, df2]
df = pd.concat(frames, ignore_index=True)

# normalizing avg_ratings with MinMax and Mean
df['minmax_norm_ratings'] = 1 + (df['avg_rating'] - df['avg_rating'].min()) / (df['avg_rating'].max() - df['avg_rating'].min()) * 9
df['mean_norm_ratings'] = 1 + (df['avg_rating'] - df['avg_rating'].mean()) / (df['avg_rating'].max() - df['avg_rating'].min()) * 9
# df['minmax_norm'] = 1 + (df['avg_rating'] - df['avg_rating'].min()) / (df['avg_rating'].max() - df['avg_rating'].min()) * 9
# df['mean_norm'] = 1 + (df['avg_rating'] - df['avg_rating'].mean()) / (df['avg_rating'].max() - df['avg_rating'].min()) * 9

display(df.head())

,url,title,authorname,avg_rating,num_rat,num_rev,total_pages,ori_year,places,series,awards,genre3,minmax_norm_ratings,mean_norm_ratings
0,https://www.goodreads.com//book/show/13496.A_G...,A Game of Thrones,George R.R. Martin,4.44,"2,084,416","55,502",835.0,2005.0,The Lands of Ice and Fire,1,"Nebula Award Nominee for Best Novel (1997), Lo...","['Fantasy', 'Fiction', 'Fantasy']",7.230769,2.641016
1,https://www.goodreads.com//book/show/30.J_R_R_...,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,4.60,"114,607","1,837",1728.0,2012.0,Middle-earth,1,NaN,"['Fantasy', 'Fiction', 'Classics']",8.021978,3.432225
2,https://www.goodreads.com//book/show/186074.Th...,The Name of the Wind,Patrick Rothfuss,4.52,"759,974","43,986",662.0,2007.0,NaN,1,Locus Award Nominee for Best First Novel and B...,"['Fantasy', 'Fiction', 'Fantasy']",7.626374,3.036621
3,https://www.goodreads.com//book/show/7235533-t...,The Way of Kings,Brandon Sanderson,4.61,"334,282","22,153",1007.0,2010.0,Roshar,1,Locus Award Nominee for Best Fantasy Novel (20...,"['Fantasy', 'Fiction', 'Fantasy']",8.071429,3.481676
4,https://www.goodreads.com//book/show/11127.The...,The Chronicles of Narnia,C.S. Lewis,4.26,"545,434","10,668",767.0,2002.0,"London, England (United Kingdom)",1,NaN,"['Fantasy', 'Classics', 'Fiction']",6.340659,1.750907


In [64]:
# Plots the normalized version of a column compared with the not normalized one

def plot_norm(not_norm_series, norm_series, norm_type: str, bins_ = 20, not_norm_color='#e67e4d', norm_color='#e67eff', edgecolor_='black'):
    
    # defining some useful variables
    # norm_min = int(round(norm_series.min()))
    # norm_max = int(round(norm_series.max()))

    norm_min = round(norm_series.min(), 2)
    norm_max = round(norm_series.max(), 2)

    
    # Set the number of subplots and the fig dimensions
    fig, ax = plt.subplots(1, 2, figsize = (15,7))

    # defining the characterics of the not-normilized graph
    ax[0].hist(not_norm_series, bins=bins_, color=not_norm_color, histtype='bar', edgecolor=edgecolor_)
    ax[0].set_title("Not normalized")
    ax[0].set_xlabel('Ratings')
    ax[0].set_ylabel('Frequency')

    # defining the characterics of the normilized graph
    ax[1].hist(norm_series, bins=bins_, color=norm_color, histtype='bar', edgecolor=edgecolor_)
    ax[1].set_title(norm_type)
    ax[1].set_xlabel(f'Ratings (normalized {norm_min} to {norm_max})')
    ax[1].set_ylabel('Frequency')

    # Set the ticks for x axis in normalized graph
    plt.sca(ax[1])
    plt.xticks(np.arange(norm_min, norm_max + 1, 1))

    fig.show()

def plot_compared_norms(first_norm, second_norm, first_norm_type: str, second_norm_type: str, bins_ = 20, \
    first_norm_color='blue',second_norm_color='darkorange', edgecolor_='white'): # F97306
    min_range = round(np.array([first_norm.min(), second_norm.min()]).min(), 2)
    max_range = round(np.array([first_norm.max(), second_norm.max()]).max(), 2)


    # Set the fig dimensions
    fig, graph = plt.subplots(figsize = (15, 8))

    graph.hist(first_norm, bins = bins_, alpha = 0.7, color=first_norm_color, edgecolor = edgecolor_, label = first_norm_type)
    graph.hist(df.mean_norm, bins = bins_, alpha = 0.7, color=second_norm_color, edgecolor = edgecolor_, label = second_norm_type)
    graph.set_title(f'Comparison between {first_norm_type} and {second_norm_type}')
    graph.legend()
    graph.set_xlabel(f'Ratings (normalized {min_range} - {max_range})')

    fig.show()

plot_norm(df.avg_rating, df.minmax_norm_ratings, norm_type='Min-Max normalization')
plot_norm(df.avg_rating, df.mean_norm_ratings, norm_type='Mean normalization')
plot_compared_norms(df.minmax_norm_ratings, df.mean_norm_ratings, first_norm_type = 'Min-Max normalization', second_norm_type='Mean normalization')



<ipython-input-64-4744d18aa437>:32: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


AttributeError: 'DataFrame' object has no attribute 'mean_norm'